## Extrae datos de idealista mediante técnica de web scrapping

### Dependencias e importaciones

In [4]:
# ! pip install ipykernel jupyter pandas numpy matplotlib seaborn requests pyodbc undetected-chromedriver requests beautifulsoup4 selenium

In [5]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import undetected_chromedriver as uc

In [6]:
URL_INICIAL_LISTADO = "https://www.idealista.com/venta-viviendas/oropesa-del-mar-castellon/con-sin-inquilinos/"
URL_INMUEBLE = "https://www.idealista.com/inmueble/%INMUEBLE_ID%/"

In [7]:
browser = uc.Chrome()

### 1º hacemos una lectura de las distintas páginas para ir anotando los ids

In [8]:
paginacion = 1
ids = []
resultados = []
# while True:
while paginacion == 1:
    print('Página: ', paginacion)
    url = f'{URL_INICIAL_LISTADO}{"pagina-" + str(paginacion) + ".htm" if paginacion > 1 else ""}'
    browser.get(url)
    time.sleep(random.randint(8, 12))

    # Trata de marcar el sí en la información de cookies
    try:
        browser.find_element("xpath", '//*[@id="didomi-notice-agree-button"]').click()
    except:
        pass

    html = browser.page_source
    soup = bs(html, 'html.parser')

    pagina_actual = int(soup.find('main', {'class': 'listing-items'}).find('div', {'class': 'pagination'}).find('li', {'class': 'selected'}).text)
    if paginacion == pagina_actual:
        # articles = soup.find('main', {'class': 'listing-items'}).find_all('article')
        articles = soup.find('main', {'class': 'listing-items'}).find_all('article', attrs={'data-element-id': True})
    else:
        break

    paginacion += 1


for article in articles:
    # ID del artículo
    element_id = article.get('data-element-id')

    # Enlace y título
    link_tag = article.find('a', class_='item-link')
    enlace = link_tag.get('href') if link_tag else ''
    titulo = link_tag.get('title') if link_tag else ''

    # Precio
    price_tag = article.find('span', class_='item-price')
    precio = price_tag.get_text(strip=True) if price_tag else ''

    # Detalles (habitaciones, m2, planta...)
    detalles_tags = article.find_all('span', class_='item-detail')
    detalles = ' | '.join([d.get_text(strip=True) for d in detalles_tags])

    # Descripción
    desc_tag = article.find('div', class_='item-description')
    descripcion = desc_tag.get_text(strip=True) if desc_tag else ''

    # Etiquetas (ej: Vistas al mar)
    tag_container = article.find('div', class_='listing-tags-container')
    etiquetas = ''
    if tag_container:
        etiquetas = ' | '.join([
            tag.get_text(strip=True) for tag in tag_container.find_all('span', class_='listing-tags')
        ])

    # Diccionario final para este artículo
    datos = {
        'id': element_id,
        'enlace': enlace,
        'titulo': titulo,
        'precio': precio,
        'detalles': detalles,
        'descripcion': descripcion,
        'etiquetas': etiquetas
    }

    resultados.append(datos)
print(resultados)




    # for article in articles:
    #     id_muebles = article.get('data-element-id')
    #     ids.append(id_muebles)
    #     print(id_muebles)
    #     time.sleep(random.randint(1, 3))
    # ids = [muebles for muebles in ids if muebles is not None]

Página:  1
[{'id': '100000581', 'enlace': '/inmueble/100000581/', 'titulo': "Piso en termalismo, Torre Bellver - La Renegá - Les Platgete'S, Oropesa del Mar", 'precio': '590.000€', 'detalles': '4 hab. | 161 m² | Planta 3ª exterior con ascensor', 'descripcion': 'Disfrutar, unos días, unas vacaciones, una temporada larga, durante todo el año, sea cual sea su elección, usted dispone de todos los servicios y más en Benicassim.\nBenicàssim cuenta con un clima y una orografía excelentes que le permiten ofrecer una amplia oferta de actividades deportivas durante todo el año. Tanto si', 'etiquetas': 'Vistas al mar'}, {'id': '103149663', 'enlace': '/inmueble/103149663/', 'titulo': "Dúplex en Forcall, 8, Torre Bellver - La Renegá - Les Platgete'S, Oropesa del Mar", 'precio': '450.000€', 'detalles': '3 hab. | 163 m² | Planta 2ª exterior con ascensor', 'descripcion': 'Disfrutar, unos días, unas vacaciones, una temporada larga, durante todo el año, sea cual sea su elección, usted dispone de todos l

In [9]:
df_resultados = pd.DataFrame(resultados)
df_resultados


,id,enlace,titulo,precio,detalles,descripcion,etiquetas
0,100000581,/inmueble/100000581/,"Piso en termalismo, Torre Bellver - La Renegá ...",590.000€,4 hab. | 161 m² | Planta 3ª exterior con ascensor,"Disfrutar, unos días, unas vacaciones, una tem...",Vistas al mar
1,103149663,/inmueble/103149663/,"Dúplex en Forcall, 8, Torre Bellver - La Reneg...",450.000€,3 hab. | 163 m² | Planta 2ª exterior con ascensor,"Disfrutar, unos días, unas vacaciones, una tem...",Vistas al mar
2,107660488,/inmueble/107660488/,"Dúplex en calle Forcall, 7, Torre Bellver - La...",395.000€,2 hab. | 163 m² | Planta 2ª exterior con ascensor,"Disfrutar, unos días, unas vacaciones, una tem...",Vistas al mar
3,105575288,/inmueble/105575288/,"Dúplex en calle Forcall, 8, Torre Bellver - La...",550.000€,4 hab. | 165 m² | Planta 2ª exterior con ascensor,Espectacular ático duplex con vistas al Mar a ...,Vistas al mar
4,103629589,/inmueble/103629589/,Casa o chalet independiente en calle Urbanizac...,925.000€,5 hab. | 375 m²,"ESPECTACULAR PROPIEDAD EN PLAYETAS, A 5 MIN A ...",Villa
5,107868849,/inmueble/107868849/,"Piso en calle dels Terrers, 1, Bahía Park - Is...",187.000€,2 hab. | 87 m² | Planta 3ª exterior con ascensor,Apartamento con vistas al mar y reforma a estr...,Con vistas | Vistas al mar
6,105995475,/inmueble/105995475/,"Piso en avenida Jardín, 57, Costa Azahar, Orop...",129.000€,2 hab. | 109 m² | Planta 8ª exterior con ascensor,"El apartamento en el edificio Jardines de Mar,...",Vistas al mar
7,102663499,/inmueble/102663499/,"Piso en Forcall, Torre Bellver - La Renegá - L...",349.000€,3 hab. | 115 m² | Planta 1ª exterior con ascensor,Espectacular apartamento con vistas al Mar a 5...,Vistas al mar
8,94575061,/inmueble/94575061/,"Casa o chalet independiente en Tombatossals, T...",775.000€,6 hab. | 365 m²,Espectacular propiedad en la exclusiva Urbaniz...,Vistas al mar | Villa
9,106411759,/inmueble/106411759/,"Piso en plaza de Mallorca, s/n, Zona Playa Mor...",390.000€,3 hab. | 128 m² | Bajo exterior con ascensor,Apartamento en Primera Línea de Playa en Urban...,Vistas al mar
